# Análisis de Ventas Históricas de Automóviles
## Parte 1: Visualización de Datos y Parte 2: Aplicación Dash

In [ ]:
# Parte 1: Visualización con Pandas, Matplotlib y Seaborn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"
df = pd.read_csv(url)
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
recession_df = df[df['Recession'] == True]
non_recession_df = df[df['Recession'] == False]

In [ ]:
# 1.1 Gráfico de líneas
df.groupby('Year')['Automobile_Sales'].sum().plot(title='Ventas por Año')
plt.ylabel('Ventas (millones)')
plt.show()

In [ ]:
# 1.2 Línea por tipo
df.groupby(['Year', 'Vehicle_Type'])['Automobile_Sales'].sum().unstack().plot()
plt.title('Ventas por tipo de vehículo')
plt.ylabel('Ventas')
plt.show()

In [ ]:
# 1.3 Seaborn recesión vs no recesión
sns.lineplot(data=df, x='Year', y='Automobile_Sales', hue='Vehicle_Type', style='Recession')
plt.title('Ventas por tipo de vehículo durante recesión')
plt.show()

In [ ]:
# 1.4 PIB en recesión y no recesión
fig, axs = plt.subplots(1, 2, figsize=(16, 6))
recession_df.groupby('Year')['GDP'].mean().plot(ax=axs[0], title="PIB durante recesión")
non_recession_df.groupby('Year')['GDP'].mean().plot(ax=axs[1], title="PIB sin recesión")
plt.show()

In [ ]:
# 1.5 Gráfico de burbujas
df['Month'] = df['Date'].dt.month
monthly_sales = df.groupby(['Month', 'Year'])['Automobile_Sales'].sum().reset_index()
plt.scatter(monthly_sales['Month'], monthly_sales['Year'], s=monthly_sales['Automobile_Sales']*10, alpha=0.5)
plt.title("Estacionalidad en ventas")
plt.xlabel("Mes")
plt.ylabel("Año")
plt.show()

In [ ]:
# 1.6 Dispersión precio vs ventas
plt.scatter(recession_df['Vehicle_Price'], recession_df['Automobile_Sales'], alpha=0.6)
plt.title("Precio vs Ventas durante recesión")
plt.xlabel("Precio medio")
plt.ylabel("Ventas")
plt.show()

In [ ]:
# 1.7 Circular gasto publicidad
ad_expenses = df.groupby('Recession')['Advertising_Expenditure'].sum()
ad_expenses.plot.pie(autopct='%1.1f%%', labels=['No Recesión', 'Recesión'], title="Gasto publicitario por recesión")
plt.ylabel('')
plt.show()

In [ ]:
# 1.8 Circular por tipo durante recesión
rec_ad_by_type = recession_df.groupby('Vehicle_Type')['Advertising_Expenditure'].sum()
rec_ad_by_type.plot.pie(autopct='%1.1f%%', title="Publicidad por tipo en recesión")
plt.ylabel('')
plt.show()

In [ ]:
# 1.9 Línea ventas y desempleo
pivot = recession_df.pivot_table(index='Date', columns='Vehicle_Type', values='Automobile_Sales', aggfunc='sum')
pivot.plot()
plt.ylabel("Ventas")
plt.title("Ventas por tipo vs desempleo durante recesión")
plt.twinx()
plt.plot(recession_df['Date'], recession_df['Unemployment_Rate'], color='black', alpha=0.4, label="Desempleo")
plt.legend()
plt.show()

## Parte 2: Dash

In [ ]:
!pip install dash

In [ ]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px

app = dash.Dash(__name__)
app.title = "Análisis de Ventas Automotrices"

app.layout = html.Div(children=[
    html.H1("Panel de Ventas de Automóviles"),
    
    dcc.Dropdown(
        id='tipo_vehiculo',
        options=[{'label': i, 'value': i} for i in df['Vehicle_Type'].unique()],
        value='Passenger Car',
        placeholder="Seleccione tipo de vehículo"
    ),

    html.Div(id='output-div', className='output-container'),
    dcc.Graph(id='ventas-vehiculo'),
    dcc.Graph(id='ventas-anuales')
])

@app.callback(
    Output('ventas-vehiculo', 'figure'),
    Output('ventas-anuales', 'figure'),
    Input('tipo_vehiculo', 'value')
)
def update_output(vehicle_type):
    df_filtered = df[df['Vehicle_Type'] == vehicle_type]
    fig1 = px.line(df_filtered, x='Date', y='Automobile_Sales', title=f"Ventas de {vehicle_type} en el tiempo")
    fig2 = px.bar(df_filtered.groupby('Year')['Automobile_Sales'].sum().reset_index(), x='Year', y='Automobile_Sales', title=f"Ventas anuales de {vehicle_type}")
    return fig1, fig2

if __name__ == '__main__':
    app.run_server(debug=False)